In [1]:
from numbers import Number
import numpy as np
import matplotlib.pyplot as plt
import onix.models.hyperfine as hyperfine

In [2]:
ground = hyperfine.states["7F0"]
excited = hyperfine.states["5D0"]
optical_hyperfine_probabilities = hyperfine.optical_hyperfine_probabilities

In [48]:
ground._Hamiltonian = ground.H_total(265.4e-4)
print((ground.energies_and_eigenstates()[0][1] - ground.energies_and_eigenstates()[0][0]) * 1e6)
print((ground.energies_and_eigenstates()[0][3] - ground.energies_and_eigenstates()[0][2]) * 1e6)

301960.9394758049
167598.8761199676


In [21]:
def print_table(lists, ndecimals=3, column_size=8, first_column_size=None):
    if first_column_size is None:
        first_column_size = column_size
    for list in lists:
        line = ""
        for kk, item in enumerate(list):
            if kk == 0:
                this_column_size = first_column_size
            else:
                this_column_size = column_size
            if isinstance(item, float):
                line += f"{item:{this_column_size}.{ndecimals}f}"
            elif isinstance(item, int):
                line += f"{item:{this_column_size}.{ndecimals}f}"
            else:
                line += f"{item:>{this_column_size}}"
        print(line)

### Optical hyperfine probabilities

In [4]:
table = [["g\\e"] + excited.states]
for g_state in ground.states:
    row = [g_state]
    for e_state in excited.states:
        row.append(optical_hyperfine_probabilities[g_state][e_state])
    table.append(row)
print("580 nm hyperfine excitation probabilities:")
print_table(table)

580 nm hyperfine excitation probabilities:
     g\e       c      c'       b      b'       a      a'
       a   0.856   0.014   0.031   0.067   0.001   0.031
      a'   0.014   0.856   0.067   0.031   0.031   0.001
       b   0.111   0.004   0.396   0.320   0.040   0.129
      b'   0.004   0.111   0.320   0.396   0.129   0.040
       c   0.005   0.011   0.149   0.036   0.322   0.478
      c'   0.011   0.005   0.036   0.149   0.478   0.322


### Ground M1 matrix elements

In [5]:
M1_elements = ground.m1_elements()
for kk, polarization in enumerate(["σ+", "σ-", "π"]):
    table = [["i\\f"] + ground.states]
    for g_state in ground.states:
        row = [g_state]
        for g_state_1 in ground.states:
            MHz_per_T_to_kHz_per_G = 1e6 / (1e3 / 1e-4)
            row.append(abs(M1_elements[kk][g_state][g_state_1] * MHz_per_T_to_kHz_per_G))
        table.append(row)
    print(f"Polarization: {polarization}. Unit: 2π kHz/G.")
    print_table(table, ndecimals=3)
    print()

Polarization: σ+. Unit: 2π kHz/G.
     i\f       a      a'       b      b'       c      c'
       a   0.480   0.014   0.086   0.036   0.031   0.025
      a'   0.014   0.480   0.080   0.227   0.039   0.048
       b   0.227   0.036   0.241   0.146   0.122   0.117
      b'   0.080   0.086   0.146   0.241   0.176   0.212
       c   0.048   0.025   0.212   0.117   0.063   0.240
      c'   0.039   0.031   0.176   0.122   0.240   0.063

Polarization: σ-. Unit: 2π kHz/G.
     i\f       a      a'       b      b'       c      c'
       a   0.480   0.014   0.227   0.080   0.048   0.039
      a'   0.014   0.480   0.036   0.086   0.025   0.031
       b   0.086   0.080   0.241   0.146   0.212   0.176
      b'   0.036   0.227   0.146   0.241   0.117   0.122
       c   0.031   0.039   0.122   0.176   0.063   0.240
      c'   0.025   0.048   0.117   0.212   0.240   0.063

Polarization: π. Unit: 2π kHz/G.
     i\f       a      a'       b      b'       c      c'
       a   0.151   0.033   0.442   0.112  

## Schiff moment sensitivities

In [24]:
table = [["Properties"] + ground.states]
energies, eigenstates = ground.energies_and_eigenstates()
row = ["Schiff sensitivity D1"]
for eigenstate in eigenstates:
    row.append(ground.Schiff_moment_sensitivity(eigenstate)[0])
table.append(row)
row = ["Schiff sensitivity D2"]
for eigenstate in eigenstates:
    row.append(ground.Schiff_moment_sensitivity(eigenstate)[1])
table.append(row)
print("7F0 Schiff moment sensitivities:")
print_table(table, ndecimals=3, first_column_size=24)
print()

7F0 Schiff moment sensitivities:
              Properties       a      a'       b      b'       c      c'
   Schiff sensitivity D1  -1.766   1.766  -0.639   0.639   0.070  -0.070
   Schiff sensitivity D2  -0.679   0.679  -0.604   0.604  -0.156   0.156

